![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 8 Notebook - AutoAI Notebook v1.19.3

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn


In [1]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install autoai-libs==1.14.13 | tail -n 1
!pip install scikit-learn==1.1.1 | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [2]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='45724efe-6afa-409a-bcc7-730915ab83a4'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/8c0d8ce9-e85f-49b8-802a-e35ca3b839ca/wml_data/37bd7ec6-7eba-4621-bf98-145a14ccd6cb/data/automl',
        model_location='auto_ml/8c0d8ce9-e85f-49b8-802a-e35ca3b839ca/wml_data/37bd7ec6-7eba-4621-bf98-145a14ccd6cb/data/automl/model.zip',
        training_status='auto_ml/8c0d8ce9-e85f-49b8-802a-e35ca3b839ca/wml_data/37bd7ec6-7eba-4621-bf98-145a14ccd6cb/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [3]:
experiment_metadata = dict(
    prediction_type='regression',
    prediction_column='Performance',
    holdout_size=0.1,
    scoring='neg_root_mean_squared_error',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=4,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    include_only_estimators=['RandomForestRegressorEstimator', 'DecisionTreeRegressorEstimator', 'RidgeEstimator', 'ExtraTreesRegressorEstimator', 'LinearRegressionEstimator', 'XGBRegressorEstimator', 'LGBMRegressorEstimator', 'SnapDecisionTreeRegressorEstimator', 'SnapRandomForestRegressorEstimator', 'SnapBoostingMachineRegressorEstimator', 'GradientBoostingRegressorEstimator'],
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='d15f38f5-c2a4-4949-8292-8b6dd6120257',
    drop_duplicates=True,
    include_batched_ensemble_estimators=[]
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [4]:
api_key = 'uCcTLZ0joXNviJfC93B5TNGKZtkoRLE51eNSDSjNK8RP'

In [5]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [6]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. 

It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`, and outliers handling:

```
from autoai_libs.utils.outliers_mitigation import remove_outliers

df = remove_outliers(df, columns=[experiment_metadata['prediction_column']])
```


In [7]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=False)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [8]:
from autoai_libs.transformers.text_transformers import TextTransformer
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA2
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from sklearn.linear_model import Ridge

#### Pre-processing & Estimator.

In [9]:
text_transformer_0 = TextTransformer(
    text_processing_options={"word2vec": {"output_dim": 20}},
    column_headers_list=[
        "Num_of_reviews", "General_rate", "Comfort", "Interior design",
        "Value for the money", "Exterior styling", "Reliability", "Year",
        "Brand", "Model", "NewTextFeature_0_word2vec(Car_name)",
        "NewTextFeature_1_word2vec(Car_name)",
        "NewTextFeature_2_word2vec(Car_name)",
        "NewTextFeature_3_word2vec(Car_name)",
        "NewTextFeature_4_word2vec(Car_name)",
        "NewTextFeature_5_word2vec(Car_name)",
        "NewTextFeature_6_word2vec(Car_name)",
        "NewTextFeature_7_word2vec(Car_name)",
        "NewTextFeature_8_word2vec(Car_name)",
        "NewTextFeature_9_word2vec(Car_name)",
        "NewTextFeature_10_word2vec(Car_name)",
        "NewTextFeature_11_word2vec(Car_name)",
        "NewTextFeature_12_word2vec(Car_name)",
        "NewTextFeature_13_word2vec(Car_name)",
        "NewTextFeature_14_word2vec(Car_name)",
        "NewTextFeature_15_word2vec(Car_name)",
        "NewTextFeature_16_word2vec(Car_name)",
        "NewTextFeature_17_word2vec(Car_name)",
        "NewTextFeature_18_word2vec(Car_name)",
        "NewTextFeature_19_word2vec(Car_name)",
    ],
    columns_to_be_deleted=[0],
    drop_columns=True,
    text_columns=[0],
)
numpy_column_selector_0 = NumpyColumnSelector(columns=[7, 8, 9])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=["int_num", "char_str", "char_str"],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[float("nan"), float("nan"), float("nan")],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
)
pipeline_0 = make_pipeline(
    text_transformer_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
text_transformer_1 = TextTransformer(
    text_processing_options={"word2vec": {"output_dim": 20}},
    column_headers_list=[
        "Num_of_reviews", "General_rate", "Comfort", "Interior design",
        "Value for the money", "Exterior styling", "Reliability", "Year",
        "Brand", "Model", "NewTextFeature_0_word2vec(Car_name)",
        "NewTextFeature_1_word2vec(Car_name)",
        "NewTextFeature_2_word2vec(Car_name)",
        "NewTextFeature_3_word2vec(Car_name)",
        "NewTextFeature_4_word2vec(Car_name)",
        "NewTextFeature_5_word2vec(Car_name)",
        "NewTextFeature_6_word2vec(Car_name)",
        "NewTextFeature_7_word2vec(Car_name)",
        "NewTextFeature_8_word2vec(Car_name)",
        "NewTextFeature_9_word2vec(Car_name)",
        "NewTextFeature_10_word2vec(Car_name)",
        "NewTextFeature_11_word2vec(Car_name)",
        "NewTextFeature_12_word2vec(Car_name)",
        "NewTextFeature_13_word2vec(Car_name)",
        "NewTextFeature_14_word2vec(Car_name)",
        "NewTextFeature_15_word2vec(Car_name)",
        "NewTextFeature_16_word2vec(Car_name)",
        "NewTextFeature_17_word2vec(Car_name)",
        "NewTextFeature_18_word2vec(Car_name)",
        "NewTextFeature_19_word2vec(Car_name)",
    ],
    columns_to_be_deleted=[0],
    drop_columns=True,
    text_columns=[0],
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29,
    ]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "float_int_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num",
    ],
    missing_values_reference_list=[float("nan")],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[float("nan")], filling_values=float("nan")
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    text_transformer_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    ],
)
ta2 = TA2(
    fun=np.add,
    name="sum",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "Num_of_reviews", "General_rate", "Comfort", "Interior design",
        "Value for the money", "Exterior styling", "Reliability", "Year",
        "Brand", "Model", "NewTextFeature_0_word2vec(Car_name)",
        "NewTextFeature_1_word2vec(Car_name)",
        "NewTextFeature_2_word2vec(Car_name)",
        "NewTextFeature_3_word2vec(Car_name)",
        "NewTextFeature_4_word2vec(Car_name)",
        "NewTextFeature_5_word2vec(Car_name)",
        "NewTextFeature_6_word2vec(Car_name)",
        "NewTextFeature_7_word2vec(Car_name)",
        "NewTextFeature_8_word2vec(Car_name)",
        "NewTextFeature_9_word2vec(Car_name)",
        "NewTextFeature_10_word2vec(Car_name)",
        "NewTextFeature_11_word2vec(Car_name)",
        "NewTextFeature_12_word2vec(Car_name)",
        "NewTextFeature_13_word2vec(Car_name)",
        "NewTextFeature_14_word2vec(Car_name)",
        "NewTextFeature_15_word2vec(Car_name)",
        "NewTextFeature_16_word2vec(Car_name)",
        "NewTextFeature_17_word2vec(Car_name)",
        "NewTextFeature_18_word2vec(Car_name)",
        "NewTextFeature_19_word2vec(Car_name)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1 = FS1(
    cols_ids_must_keep=range(0, 30),
    additional_col_count_to_keep=20,
    ptype="regression",
)
ridge = Ridge(
    alpha=0.5102628927202548,
    copy_X=False,
    max_iter=970,
    random_state=33,
    tol=0.005409847466878582,
)


#### Pipeline.

In [10]:
pipeline = make_pipeline(union, numpy_permute_array, ta2, fs1, ridge)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [11]:
from sklearn.metrics import make_scorer

from autoai_libs.scorers.scorers import neg_root_mean_squared_error

scorer = make_scorer(neg_root_mean_squared_error)

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [12]:
pipeline.fit(train_X.values, train_y.values.ravel());

/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=4.39941e-08): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [13]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

-0.21959620525128576


In [14]:
pipeline.predict(test_X.values[:5])

array([4.8271017, 2.9624271, 4.16173  , 4.497771 , 4.9379773],
      dtype=float32)

<a id="saving"></a>
## Store the model

In this section you will learn how to store the trained model.

In [15]:
model_metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'Trained AutoAI pipeline'
}

stored_model_details = wml_client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

Inspect the stored model details.

In [16]:
stored_model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Performance',
  'pipeline': {'id': '89ca5f4e-4413-45b2-a8ff-419352bf1a54'},
  'schemas': {'input': [{'fields': [{'name': 'Car_name', 'type': 'other'},
      {'name': 'Num_of_reviews', 'type': 'double'},
      {'name': 'General_rate', 'type': 'double'},
      {'name': 'Comfort', 'type': 'double'},
      {'name': 'Interior design', 'type': 'double'},
      {'name': 'Value for the money', 'type': 'double'},
      {'name': 'Exterior styling', 'type': 'double'},
      {'name': 'Reliability', 'type': 'double'},
      {'name': 'Year', 'type': 'integer'},
      {'name': 'Brand', 'type': 'other'},
      {'name': 'Model', 'type': 'other'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '8c1a58c6-62b5-4dc4-987a-df751c2756b6',
   'name': 'hybrid_0.1'},
  'training_data_references': [{'connection': {},
    'location': {'bucket': 'carratesinnewyork-donotdelete-pr-cpvsxjzhra0j6l',
     'href': 

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___